In [ ]:
!pip3 install azure-storage-blob # Microoft Azure
!pip3 install pyarrow # Apache Arrow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.5/394.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00


In [ ]:
# Import libraries
from io import StringIO
import numpy as np
import pandas as pd
import json
import requests
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
#Extracting from datasource
URL = "https://data.wa.gov/resource/f6w7-q2d2.json"
total_rows = 0
df_raw = pd.DataFrame()

while total_rows < 7500: #set while loop to pass through 1000 rows limit
    response = requests.get(URL, params={"$offset": total_rows})
    data = response.json()
    df_raw = pd.concat([df_raw, pd.DataFrame(data)], ignore_index=True) #pd.concat don't use df_raw.append
    total_rows += len(data)

print(df_raw.info())
print(df_raw.shape)
print(df_raw.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   vin_1_10                     8000 non-null   object
 1   county                       8000 non-null   object
 2   city                         8000 non-null   object
 3   state                        8000 non-null   object
 4   zip_code                     8000 non-null   object
 5   model_year                   8000 non-null   object
 6   make                         8000 non-null   object
 7   model                        8000 non-null   object
 8   ev_type                      8000 non-null   object
 9   cafv_type                    8000 non-null   object
 10  electric_range               8000 non-null   object
 11  base_msrp                    8000 non-null   object
 12  legislative_district         8000 non-null   object
 13  dol_vehicle_id               8000

In [12]:
# how do i hide this key? I can't push onto github with key visible
CONNECTION_STRING_AZURE_STORAGE = "connectionkey"
CONTAINER_AZURE = 'evinfra'
blob_name = "evinfra_20240402.csv"
# Convert DataFrame to CSV
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

# Create the BlobServiceClient object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

# Get a blob client using the container name and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE, blob=blob_name)

# Upload the CSV data
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

ValueError: Connection string is either blank or malformed.